# 列舉所有資產的riskiness r

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [3]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [4]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [86]:
def get_top10_etf(data_file):
    # 讀檔
    #data_file = '../../HW1/ETF_data_set/return_data/Alternatives ETF List (35)/Alternatives ETF List (35)_return.csv';
    df = pd.read_csv(data_file)
    etf_list = list(df.columns)[2:]
    df = df.iloc[:,2:]
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        arr_returns = df[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        
        #################################################
        #test
        
        arr_returns = arr_returns[:36] #< 前36個月 要和後面選擇的return連動
        
        
        
        
        ####################################################
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前10名
    top10 = sorted_x[:5]
        # top10 = sorted_reversed_x[:10]
    #print(top10)
    
    # 取前10的ETF名稱
    etf_top10 = [etf[0] for etf in top10]
    
    #前10的平均
    top10_average = df[etf_top10].mean(axis=1)
    
    #print(etf_top10)
    return etf_top10 , top10_average

# 讀取全部ETF資料夾

In [105]:
group_top5 = dict()
data_path = '../../FinalProject/2014to2018RE_data/'
allgroup_top10_return = DataFrame()
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        #print(file_path)
        if os.path.isfile(file_path):
            # 提出[前10名的名稱,前10名的平均各期報酬]
            [ top10_etf,allgroup_top10_return[subdir] ] = get_top10_etf(file_path)
            group_top5[subdir]=top10_etf
            df = pd.read_csv(file_path)
            date_list = df['Date']
            print('Group:',subdir)
            print('Top10:',top10_etf)
            print('================')
allgroup_top10_return = pd.concat([date_list,allgroup_top10_return],axis=1)

###############################################
#要區分出是依照哪個區間找出來的top5 etf 只有在那個區間的報酬才拿來用

allgroup_top10_return = allgroup_top10_return.iloc[:36,:]

###############################################

Group: Alternatives ETF List (35)
Top10: ['FLAG', 'MOM', 'SIZ', 'CHEP', 'WTMF']
Group: Broad Asia ETF List (44)
Top10: ['VXUS', 'AIA', 'FPA', 'AXJV', 'FPXI']
Group: Commodity ETF List (125)
Top10: ['UCO', 'BOIL', 'DTO', 'SCO', 'LD']
Group: Consumer Discretionary Equity ETF List (31)
Top10: ['XLY', 'RXI', 'FDIS', 'RCD', 'VCR']
Group: Consumer Staples Equity ETF List (26)
Top10: ['KXI', 'PSCC', 'XLP', 'FSTA', 'VDC']
Group: Crude Oil ETF List (22)
Top10: ['UCO', 'DTO', 'SCO', 'SZO', 'DBO']
Group: Currency ETF List (36)
Top10: ['USDU', 'FXE', 'DBV', 'FXA', 'FXS']
Group: Developed Asia Pacific ETF List (108)
Top10: ['IDOG', 'DOO', 'VPL', 'EWJ', 'FDT']
Group: Developed Markets ETF List 1 (100)
Top10: ['IDOG', 'VSS', 'VT', 'URTH', 'KXI']
Group: Developed Markets ETF List 2 (100)
Top10: ['EFZ', 'DOO', 'PIZ', 'ACIM', 'EXI']
Group: Developed Markets ETF List 3 (93)
Top10: ['CROP', 'PEX', 'FTAG', 'EDOM', 'FID']
Group: Emerging Asia Pacific ETF List (114)
Top10: ['YINN', 'INDL', 'FXP', 'INCO', 'SM

In [106]:
allgroup_top10_return

,Date,Alternatives ETF List (35),Broad Asia ETF List (44),Commodity ETF List (125),Consumer Discretionary Equity ETF List (31),Consumer Staples Equity ETF List (26),Crude Oil ETF List (22),Currency ETF List (36),Developed Asia Pacific ETF List (108),Developed Markets ETF List 1 (100),...,Municipal Bond ETF List (29),Preferred Stock ETF List (12),Real Estate ETF List (48),Target Maturity Date Corporate Bond ETF List (24),Technology Equity ETF List (75),Telecom Equity ETF List (10),Total Bond Market ETF List (82),Treasuries ETF List (51),Utilities Equity ETF List (24),Volatility ETF List (18)
0,2014-01-01,-0.006256,-0.062147,0.028862,-0.055191,-0.047654,-0.002994,-0.012675,-0.045499,-0.039184,...,0.028300,0.046377,0.030040,0.029735,-0.022945,-0.005846,0.026794,-0.011839,0.028193,0.104873
1,2014-02-01,0.008194,0.055428,0.006687,0.066379,0.038894,-0.024239,0.012926,0.054542,0.058969,...,0.006666,0.020604,0.049040,0.007742,0.046122,0.011243,0.003333,-0.001868,0.043021,-0.094300
2,2014-03-01,0.003106,0.007184,-0.023134,-0.027083,0.023899,0.005284,0.012210,-0.003043,0.005157,...,0.002483,0.016984,-0.000495,0.002463,0.001761,0.031528,0.003039,0.002147,0.037247,-0.026063
3,2014-04-01,0.007033,0.016429,0.046335,-0.014606,0.020055,0.000538,-0.002046,0.008422,0.016996,...,0.011820,0.019594,0.039694,0.013616,-0.002357,-0.012493,0.011256,-0.000911,0.045796,-0.042291
4,2014-05-01,-0.000224,0.032196,-0.045340,0.025536,0.015818,-0.015302,-0.008007,0.024618,0.019894,...,0.010381,0.012940,0.027770,0.018185,0.034202,0.036021,0.009493,-0.007963,0.000197,-0.110062
5,2014-06-01,0.008723,0.003898,-0.015257,0.018926,-0.002486,-0.011966,0.002942,0.011368,0.003643,...,-0.000718,0.005194,0.002871,0.004069,0.023891,0.010202,-0.000259,-0.000708,0.041641,-0.094453
6,2014-07-01,-0.012731,0.020553,-0.013534,-0.014921,-0.035181,0.027495,-0.011448,-0.003658,-0.020789,...,0.003418,-0.004730,0.006481,0.001339,0.011663,0.001070,-0.003366,0.000079,-0.058099,0.066388
7,2014-08-01,0.005711,0.023964,0.018236,0.039493,0.048832,0.007994,-0.002133,-0.004305,0.019199,...,0.011224,0.019842,0.023285,0.014178,0.037791,0.011847,0.014367,-0.007011,0.052857,-0.085349
8,2014-09-01,-0.001115,-0.057304,-0.005039,-0.031909,-0.003973,0.008842,-0.021874,-0.042999,-0.042657,...,0.001203,-0.012993,-0.065185,-0.006558,-0.016656,-0.025836,-0.005849,0.005484,-0.038272,0.062731
9,2014-10-01,-0.001647,0.009467,0.010723,0.024859,0.038746,0.042187,-0.002852,0.006029,0.003579,...,0.004938,0.015318,0.098709,0.012277,0.014902,0.025744,0.008017,0.000618,0.080558,-0.081151


# 將allgroup_top5 先存取成CSV

In [107]:
allgroup_top10_return.to_csv("allgroup_top5_return.csv")

# 再次計算Riskiness R，並依照R值排序，傳回前五名的Group

In [113]:
def get_top5_Group(data_file):

    df1 = pd.read_csv(data_file)
    etf_list1 = list(df1.columns)[2:]
    df1 = df1.iloc[:,2:]

    etf_dic1 = {}
    for etf_name in etf_list1:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        arr_returns1 = df1[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        guess1 = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic1[etf_name] = get_riskiness_r(guess1,arr_returns1)
    
    
    sorted_x1 = sorted(etf_dic1.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前5名
    top5 = sorted_x1[:5]
        # top5 = sorted_reversed_x[:5]
    #print(top5)
    
    # 取前5的ETF名稱
    etf_top5 = [etf[0] for etf in top5]

    df2 = df1[etf_top5]
    
    return df2

In [10]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [11]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top10_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top10_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [12]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

# 依次計算32組的權重

In [33]:
a = list(range(0, 32))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)
C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738875873
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938272427682
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938273561566
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738560733
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0

權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382737570457
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.74938275550922
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125] 誤差值: 30.749382738476218
權重: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 

# 計算出group中 前五名的權重

In [114]:
df2 = get_top5_Group("C:\\Users\\USER\\HW1--data-mining\\FinalProject\\Scripts\\allgroup_top5_return.csv")

print("=============")
print("Top_5_Group")
print(df2.columns)

Top_5_Group
Index(['Total Bond Market ETF List (82)', 'Real Estate ETF List (48)',
       'Developed Asia Pacific ETF List (108)',
       'Developed Markets ETF List 1 (100)', 'Emerging Markets ETF List (79)'],
      dtype='object')


In [119]:
a = list(range(0, 5))
all_return = df2.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699311450244
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699311450244
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699311450244
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699277772157
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699336919056
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699334164806
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699047231302
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482698906390747


C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


權重: [-2.33841759  1.32100946  1.05216177  0.46625234  0.49899401] 誤差值: 100000.91648598296
權重: [-0.42525488 -0.35552826 -0.16480408  0.32225977  0.35361388] 誤差值: 100127.83470031185
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699277772157
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699336919056
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699334164806
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482699047231302
權重: [0.2 0.2 0.2 0.2 0.2] 誤差值: 1.9482698906390747
權重: [-2.33841759  1.32100946  1.05216177  0.46625234  0.49899401] 誤差值: 100000.91648598296
權重: [-0.42525488 -0.35552826 -0.16480408  0.32225977  0.35361388] 誤差值: 100127.83470031185
權重: [-0.29782767  0.1662529   0.2511373   0.16841529  0.15279781] 誤差值: 100008.06235588087
權重: [0.28210117 0.1174014  0.17406351 0.21029503 0.16128151] 誤差值: 2.018931141290235
權重: [0.16493307 0.23530182 0.2300719  0.21536279 0.21290416] 誤差值: 1.7702737729124731
權重: [0.20373965 0.19660108 0.20382353 0.20158371 0.19866551] 誤差值: 1.9485954200043796
權重: [0.18497216 0.21614284 0.21770658 0.208823

權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306473835821
權重: [0.14212187 0.05989359 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306460799412
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306467099904
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062485] 誤差值: 0.6057306651414192
權重: [0.14212187 0.05989358 0.12132691 0.39664529 0.37062486] 誤差值: 0.6057306265810619
權重: [0.1421187  0.05989179 0.12130046 0.39664438 0.37063038] 誤差值: 0.5921370041775393
[0.14212187 0.05989358 0.12132691 0.39664529 0.37062485]


C:\Users\USER\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


# 計算出以最事權重購買之後的報酬

## 取出TOP5 group 裡面選出的 TOP5 ETF 
## 並且計算其Return 後回傳績效表

In [109]:
top5_group_name = df2.columns.tolist()
top5_etf_name = []
for name in top5_group_name :
    print('group is',name)
    print('ETF is ',group_top5[name])
    

group is Total Bond Market ETF List (82)
ETF is  ['NEAR', 'ULST', 'MINT', 'XMPT', 'BLV']
group is Real Estate ETF List (48)
ETF is  ['ICF', 'RWR', 'WPS', 'FRI', 'SCHH']
group is Developed Asia Pacific ETF List (108)
ETF is  ['IDOG', 'DOO', 'VPL', 'EWJ', 'FDT']
group is Developed Markets ETF List 1 (100)
ETF is  ['IDOG', 'VSS', 'VT', 'URTH', 'KXI']
group is Emerging Markets ETF List (79)
ETF is  ['EEV', 'EDZ', 'EDC', 'EDOG', 'EMQQ']


In [110]:
top5_group_name

['Total Bond Market ETF List (82)',
 'Real Estate ETF List (48)',
 'Developed Asia Pacific ETF List (108)',
 'Developed Markets ETF List 1 (100)',
 'Emerging Markets ETF List (79)']

In [111]:
data_path = 'C:\\Users\\USER\\github\\HW1--data-mining\\FinalProject\\2014to2018RE_data'
best_wtop5_return = DataFrame()
Return_value = {}
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        for name in top5_group_name : ##### 找出group位置
            if name in file_path :
                print(file_path)
                df = pd.read_csv(file_path)  ###### df = Group中全部ETF之RETURN
                value = 0
                for ETF in group_top5[name] : ##### 找出top5group中被選出來的ETF
                    #########################################  和前面連動  找地36個位置的return
                    value += df[ETF].iloc[36]
                    dis = len(group_top5[name])
                Return_value[name] = value/dis


C:\Users\USER\github\HW1--data-mining\FinalProject\2014to2018RE_data\Developed Asia Pacific ETF List (108)\Developed Asia Pacific ETF List (108)_return.csv
C:\Users\USER\github\HW1--data-mining\FinalProject\2014to2018RE_data\Developed Markets ETF List 1 (100)\Developed Markets ETF List 1 (100)_return.csv
C:\Users\USER\github\HW1--data-mining\FinalProject\2014to2018RE_data\Emerging Markets ETF List (79)\Emerging Markets ETF List (79)_return.csv
C:\Users\USER\github\HW1--data-mining\FinalProject\2014to2018RE_data\Real Estate ETF List (48)\Real Estate ETF List (48)_return.csv
C:\Users\USER\github\HW1--data-mining\FinalProject\2014to2018RE_data\Total Bond Market ETF List (82)\Total Bond Market ETF List (82)_return.csv


In [118]:
(Return_value * r2 ).mean()



0.0035422777492903497

In [116]:
###### 2017年 1/1日的 return

pro

0.02071736170168935

In [117]:
Return_value * r2

array([ 6.55967479e-03,  2.05416726e-03, -9.74917439e-05,  4.92329867e-03,
        4.27173977e-03])

In [ ]:
data_path = '../../HW1/ETF_data_set/new_ETF_monthly/'
best_wtop5_return = DataFrame()
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        #print(file_path)